In [20]:
import pandas as pd
from os import path

#directory = "results-gtm"
directory = "results-sgtm"
#directory = "results-iter1"
columns = ["label", "prediction", "url", "visit_id"]
df = None


# I ran 10-fold cross validation so there are 10 different files for test data predictions
for i in range(10):
    # The ML model output files use a weird separator: " |$| "
    temp_df = pd.read_table(path.join(directory, f"tp_{i}"), header=None, sep="\ \|\$\|\ ", names=columns)

    if df is None:
        df = temp_df
    else:
        df = pd.concat([df, temp_df])


/home/siim/WebGraph/.venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [11]:
fn = df[df["prediction"] & ~df["label"]] #.to_csv("temp_df.csv")
#fn = df[df["prediction"] & ~df["label"] & df["url"].str.contains("/j/collect") ] #.to_csv("temp_df.csv")

In [12]:
from IPython.core.display_functions import display

fn.to_csv("temp_df.csv")

In [27]:
def is_google(df):
    google_domains = ['region1.google-analytics.com', 'region1.analytics.google.com', 'stats.g.doubleclick.net', 'www.google-analytics.com', 'analytics.google.com']
    values = df["url"].str.contains("|".join(google_domains), regex=True)
    return values

df_sgtm = df[df["url"].str.contains('/g/collect') & ~ is_google(df)]
df_sgtm.to_csv("temp_df.csv")

AttributeError: 'Series' object has no attribute 'values_count'

In [30]:
df_sgtm["prediction"].value_counts()


False    160
True       4
Name: prediction, dtype: int64